# ANÁLISIS DATAFRAME 'DIAMONDS' CON PYSPARK.

## OBJETIVO DEL ESTUDIO.

<p>El objetivo del estudio es crear un modelo que sea capaz de predecir el precio de un diamante.</p>

## 1.-CARGA DE LIBRERÍAS.

In [1]:
import seaborn as sns
import pandas as pd
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, StringType, NumericType, IntegerType
from pyspark.sql.functions import col, sum
from pyspark.ml.feature import StringIndexer, Imputer, OneHotEncoder, VectorAssembler, RobustScaler, MinMaxScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import RandomForestClassifier, MultilayerPerceptronClassifier
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

## 2.-CREACIÓN SESIÓN PYSPARK.

In [2]:
spark = SparkSession.builder.appName("pipeline_diamonds").getOrCreate()

## 3.-CARGA DEL DATAFRAME.

In [3]:
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/refs/heads/master/diamonds.csv' # Se define la URL del archivo csv.
csv_path = 'diamonds.csv'           # Se define el nombre del archivo csv.

with open(csv_path, 'wb') as file:                  # Se descarga el archivo csv.
    file.write(requests.get(url).content)
    
schema = StructType([                               # Se define el Schema del DataFrame.
    StructField('carat', FloatType(), True),
    StructField('cut', StringType(), True),
    StructField('color', StringType(), True),
    StructField('clarity', StringType(), True),
    StructField('depth', FloatType(), True),
    StructField('table', FloatType(), True),
    StructField('price', FloatType(), True),
    StructField('x', FloatType(), True),
    StructField('y', FloatType(), True),
    StructField('z', FloatType(), True)
])

# Se lee el archivo csv con el Schema definido.
df = spark.read.csv(csv_path,
                    header=True,        # Se indica que la primera fila es el encabezado.
                    inferSchema=False,  # Se indica que no se infiera el tipo de dato de cada columna.
                    schema=schema       # Se indica el Schema definido.
                    )
df.show(5)          # Se muestran los primeros 5 registros del DataFrame.
df.printSchema()    # Se muestra el Schema del DataFrame.

+-----+-------+-----+-------+-----+-----+-----+----+----+----+
|carat|    cut|color|clarity|depth|table|price|   x|   y|   z|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
| 0.23|  Ideal|    E|    SI2| 61.5| 55.0|326.0|3.95|3.98|2.43|
| 0.21|Premium|    E|    SI1| 59.8| 61.0|326.0|3.89|3.84|2.31|
| 0.23|   Good|    E|    VS1| 56.9| 65.0|327.0|4.05|4.07|2.31|
| 0.29|Premium|    I|    VS2| 62.4| 58.0|334.0| 4.2|4.23|2.63|
| 0.31|   Good|    J|    SI2| 63.3| 58.0|335.0|4.34|4.35|2.75|
+-----+-------+-----+-------+-----+-----+-----+----+----+----+
only showing top 5 rows

root
 |-- carat: float (nullable = true)
 |-- cut: string (nullable = true)
 |-- color: string (nullable = true)
 |-- clarity: string (nullable = true)
 |-- depth: float (nullable = true)
 |-- table: float (nullable = true)
 |-- price: float (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- z: float (nullable = true)



## 4.-PRE-PROCESADOS.

### 4.1.-Preparación previa del DataFRame.

#### 4.1.1.-Eliminación de valores 'NaN' en la columna objetivo.

In [4]:
# La variable objetivo es 'precio'. Por tanto, se eliminan las filas con valores nulos en esta variable.
df_regresion = df.dropna(subset=['price'])

In [5]:
# La variable objetivo es 'cut'. Por tanto, se eliminan las filas con valores nulos en esta variable.
df_clasificacion = df.dropna(subset=['cut'])

#### 4.1.2.-Contabilizar valores 'NaN' en las columnas del DataFrame.

In [6]:
# Se cuentan los valores nulos en cada columna.
df_regresion.select([sum(col(c).isNull().cast('int')).alias(c) for c in df.columns]).show()

+-----+---+-----+-------+-----+-----+-----+---+---+---+
|carat|cut|color|clarity|depth|table|price|  x|  y|  z|
+-----+---+-----+-------+-----+-----+-----+---+---+---+
|    0|  0|    0|      0|    0|    0|    0|  0|  0|  0|
+-----+---+-----+-------+-----+-----+-----+---+---+---+



In [7]:
# Se cuentan los valores nulos en cada columna.
df_clasificacion.select([sum(col(c).isNull().cast('int')).alias(c) for c in df.columns]).show()

+-----+---+-----+-------+-----+-----+-----+---+---+---+
|carat|cut|color|clarity|depth|table|price|  x|  y|  z|
+-----+---+-----+-------+-----+-----+-----+---+---+---+
|    0|  0|    0|      0|    0|    0|    0|  0|  0|  0|
+-----+---+-----+-------+-----+-----+-----+---+---+---+



<p>Aunque ninguna de las columnas presentan valores nulos, se va a considerar (a efectos de ejemplo del ejercicio) que sí los tiene.</p>

## 5.-REGRESIÓN DE LA COLUMNA NUMÉRICA 'price'.

<p>Dado que el ejercicio se compone de dos partes bien diferenciadas: regresión de la columna 'price' y clasificación de la columna 'carat', y para evitar posibles problemas con la fuente de datos, se crea una copia del DataFrame por cada tipo de modelado.</p>

### 5.1.-Separación de las columnas numéricas, no-numéricas y variable objetivo.

In [8]:
# Se seleccionan los nombres de las columnas a las que aplicar Preprocesados.
num_cols = [field.name for field in df_regresion.schema.fields if isinstance(field.dataType, NumericType)]
cat_cols = [field.name for field in df_regresion.schema.fields if isinstance(field.dataType, StringType) and field.name != 'price']
label_col = 'price'

### 5.2.-Pre-Procesados.

#### 5.2.1.-Indexados.

##### 5.2.1.1.-Indexado de la columna objetivo: 'price'.

In [9]:
# Se indexa la columna a predecir 'price' como columna objetivo ('label').
indexer_label = StringIndexer(
    inputCol=label_col,         # Nombre de la columna objetivo a indexar.
    outputCol='label',          # Nombre de salida de la columna indexada.
    handleInvalid='keep'        # Se mantienen los valores no vistos en el entrenamiento.
)

#### 5.2.1.2.-Indexado de la columnas categóricas.

In [10]:
# Se indexan las columnas categóricas de la entrada (features) que no son la columna label a predecir.
# Se crea un objeto StringIndexer por cada columna categórica a indexar.
indexers_features = [
    StringIndexer(inputCol=c,                   # Nombre de la columna categórica a indexar.
                  outputCol=c + '_indexed',     # Nombre de salida de la columna indexada.
                  handleInvalid='keep'          # Se mantienen los valores no vistos en el entrenamiento.
                  ) for c in cat_cols
]
cat_cols_indexed = [c + '_indexed' for c in cat_cols]   # Se crea una lista con los nombres de las columnas indexadas.
print(cat_cols_indexed)

['cut_indexed', 'color_indexed', 'clarity_indexed']


#### 5.2.2.-Imputers.

##### 5.2.2.1.-Imputers de variables numéricas.

In [11]:
imputer_numericas = Imputer(
    inputCols=num_cols,                                 # Nombre de las columnas numéricas a imputar.
    outputCols=[c + '_imputed' for c in num_cols],      # Nombre de salida de las columnas imputadas
    strategy='median'                                   # Se imputa con la mediana.
)
num_cols_imputed = [c + '_imputed' for c in num_cols]   # Se crea una lista con los nombres de las columnas imputadas.
print(num_cols_imputed)

['carat_imputed', 'depth_imputed', 'table_imputed', 'price_imputed', 'x_imputed', 'y_imputed', 'z_imputed']


##### 5.2.2.2.-Imputers de variables categóricas.

In [12]:
imputer_categoricas = Imputer(
    inputCols=cat_cols_indexed,                             # Nombre de las columnas categóricas a imputar.
    outputCols=[c + '_imputed' for c in cat_cols_indexed],  # Nombre de salida de las columnas imputadas.
strategy='mode'                                             # Se imputa con la moda.
)
cat_cols_indexed_imputed = [c + '_imputed' for c in cat_cols_indexed]   # Se crea una lista con los nombres de las columnas imputadas.
print(cat_cols_indexed_imputed)

['cut_indexed_imputed', 'color_indexed_imputed', 'clarity_indexed_imputed']


##### 5.2.2.3.-Codificación de variables categóricas.

In [13]:
encoder_onehot = [
    OneHotEncoder(inputCol=c, outputCol=c + '_onehot') 
    for c in cat_cols_indexed_imputed
]
cat_cols_onehot = [c + '_onehot' for c in cat_cols_indexed_imputed]
print(cat_cols_onehot)

['cut_indexed_imputed_onehot', 'color_indexed_imputed_onehot', 'clarity_indexed_imputed_onehot']


#### 5.2.3.-Escalados.

<p>En el ejercicio se va a utilizar el algoritmo RandomForestRegressor para generar el modelo de regresión. Este algoritmo, al no estar basado distancias, no es tan sensible a las diferentes escalas de los valores de las variables. No obstante, a modo de ejemplo, y de cara a realizar un ejercicio más completo, se va a efectuar un escalado.</p>
<p>El método del escalado que se va a emplear es RobustScaler (en lugar de lo indicado en el enunciado: MinMaxScaler o StandardScaler), debido a que, por lo desarrollado en el ejercicio del módulo 3, en el DataFrame de 'Diamonds' existen columnas con distribuciones muy asimétricas y con presencia de numerosos outliers. RobustScaler es más apropiado para estos casos, ya que hace uso de la mediana y los rangos inter-cuartílicos.</p>

##### 5.2.3.1-Ensamblado de variables.

<p>Antes de aplicar el escalado, se aplica el ensamblado de las variables para unificar ambos tipos de variables, numéricas y categóricas, en un único vector.</p>

In [14]:
# Se crea una lista con el nombre de las columnas a ensamblar, que seran las columnas 'features'.
col_ensambladas = [col for col in num_cols_imputed] + \
                         [col for col in cat_cols_onehot]
print(col_ensambladas)

['carat_imputed', 'depth_imputed', 'table_imputed', 'price_imputed', 'x_imputed', 'y_imputed', 'z_imputed', 'cut_indexed_imputed_onehot', 'color_indexed_imputed_onehot', 'clarity_indexed_imputed_onehot']


In [15]:
# Se ensamblan los nombres de las columnas numéricas y categóricas en una única columna 'features'.
assembler = VectorAssembler(
    inputCols=col_ensambladas,      # Nombre de las columnas a ensamblar.
    outputCol='features'            # Nombre de salida de la columna ensamblada.
)

##### 5.2.3.2-Escalado.

In [16]:
# Escalado de todas las 'features' utilizando RobustScaler.
scaler = RobustScaler(inputCol="features", outputCol="scaled_features")

### 5.3.-Modelado, Pipelines y Evaluación.

#### 5.3.1.-Generación del modelo.

<p>Como se ha mencionado anteriormente, para la regresión se utiliza el algoritmo RandomForestRegressor.</p>

In [17]:
# Se eligen parámetros de ejemplo (100 árboles, profundidad máxima de 5)
model_regresion = RandomForestRegressor(featuresCol="scaled_features",   # Vector de características.
                                    labelCol=label_col,                 # Columna objetivo.
                                        numTrees=100,                   # Número de árboles.
                                        maxDepth=5                      # Profundidad máxima.
                                        )

#### 5.3.2.-Particionamiento de datos.

In [18]:
df_reg_train, df_reg_test = df_regresion.randomSplit([0.8, 0.2], seed=42)  # Se divide el DataFrame en Train y Test.

#### 5.3.3.-Generación del Pipeline.

In [19]:
pipeline_regresion = Pipeline(stages = [
    indexer_label,          # Indexador de la columna objetivo.
    *indexers_features,     # Indexadores de las columnas categóricas. (se coloca '*' para desempaquetar la lista de objetos).
    imputer_numericas,      # Imputador de las columnas numéricas.
    imputer_categoricas,    # Imputador de las columnas categóricas.
    *encoder_onehot,        # OneHotEncoder de las columnas categóricas. (se coloca '*' para desempaquetar la lista de objetos).
    assembler,              # Ensamblador de las columnas numéricas y categóricas.
    scaler,                 # Escalador de las 'features'.
    model_regresion         # Modelo de Regresión.
])

#### 5.3.4.-Entrenamiento del modelo.

In [20]:
model_reg = pipeline_regresion.fit(df_reg_train)  # Se entrena el modelo.

#### 5.3.5.-Predicción del modelo.

In [21]:
prediccion_reg = model_reg.transform(df_reg_test)  # Se realiza la predicción.

#### 5.3.6.-Evaluación del modelo.

In [22]:
evaluador_r2 = RegressionEvaluator(labelCol=label_col, predictionCol='prediction', metricName='r2')
r2 = evaluador_r2.evaluate(prediccion_reg)
evaluador_mae = RegressionEvaluator(metricName='mae')
mae = evaluador_mae.evaluate(prediccion_reg) 
evaluador_mse = RegressionEvaluator(metricName='mse')
mse = evaluador_mse.evaluate(prediccion_reg)
evaluador_rmse = RegressionEvaluator(labelCol=label_col, predictionCol='prediction', metricName='rmse')
rmse = evaluador_rmse.evaluate(prediccion_reg)

print("=== Evaluación del Modelo de Regresión (RandomForestRegressor con RobustScaler) ===")
print(f"R2: {r2}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

=== Evaluación del Modelo de Regresión (RandomForestRegressor con RobustScaler) ===
R2: 0.9773520052305722
MAE: 1981.4725284922758
MSE: 7849110.502337356
RMSE: 607.0337566183553


Del resultado de la evaluación del modelo de regresión podemos concluir lo siguiente:
* R2: El valor de esta métrica indica que el 97,7% de la variabilidad en el precio se explica por el modelo. Es un valor realmente bueno, ya que casi toda la variabilidad de los datos se captura con la infdormación empleada.
* MAE: En promedio, las predicciones se desvían del valor real en cerca de 1989.81 unidades de precio. 
* MSE: Esta métrica penaliza más fuertemente los errores grandes, ya que eleva al cuadrado cada diferencia. Un MSE elevado puede indicar la presencia de algunos errores significativamente grandes o outliers. Este valor, sin embargo, está en unidades al cuadrado, lo que dificulta una interpretación directa en el contexto del precio.
* RMSE: Esta métrica es la raíz cuadrada del MSE. El valor 610,975$ indica que las predicciones del modelo pueden diferir en la cantidad indicada por la métrica. Este valor (610,975$), teniendo en cuenta que el valor mínimo de la columna 'price' es 326,0$, el valor máximo es 18.823,0$, media de 3.932,0$, mediana 2401,0$ y que la distribución de precios presenta numerosos outliers y tiene una clara y acusada asimetría hacia valores altos, se podría considerar que el RMSE de 610,975 $ podría ser un valor aceptable. Sin embargo, si se desea mejorar el resultado, se podría actuar sobre los siguientes factores:
    - Tratamiento de outliers: Aunque se ha utilizado el escalado RobustScaler (recomendado ante la presencia de outliers), se podrían tratar, previas al escalado, los outliers mediante otras técnicas (Valla de Tukey, Z-Score, etc.)
    - Transformación de los valores de la columna objetivo mediante funciones raíz o logarítmica.
    - Ajuste de hiperparámetros del modelo.
    - Evaluación y utilización de otros modelos de regresión.


## 6.-REGRESIÓN DE LA COLUMNA CATEGÓRICA 'cut'.

<p>Esta es la fase del modelado de clasificación de la columna 'cut'.</p>

### 6.1.-Separación de las columnas numéricas, no-numéricas y variable objetivo.

In [23]:
# Se seleccionan los nombres de las columnas a las que aplicar Preprocesados.
num_cols_clas = [field.name for field in df_clasificacion.schema.fields if isinstance(field.dataType, NumericType)]
cat_cols_clas = [field.name for field in df_clasificacion.schema.fields if isinstance(field.dataType, StringType) and field.name != 'cut']
label_col_clas = 'cut'

### 6.2.-Pre-Procesados.

#### 6.2.1.-Indexados.

##### 6.2.1.1.-Indexado de la columna objetivo: 'cut'.

In [24]:
# Se indexa la columna a predecir 'cut' como columna objetivo ('label').
indexer_label_clas = StringIndexer(
    inputCol=label_col_clas,        # Nombre de la columna objetivo a indexar.
    outputCol='label',              # Nombre de salida de la columna indexada.
    handleInvalid='keep'            # Se mantienen los valores no vistos en el entrenamiento.
)

#### 6.2.1.2.-Indexado de la columnas categóricas.

In [25]:
# Se indexan las columnas categóricas de la entrada (features) que no son la columna label a predecir.
# Se crea un objeto StringIndexer por cada columna categórica a indexar.
indexers_features_clas = [
    StringIndexer(inputCol=c,                   # Nombre de la columna categórica a indexar.
                  outputCol=c + '_indexed',     # Nombre de salida de la columna indexada.
                  handleInvalid='keep'          # Se mantienen los valores no vistos en el entrenamiento.
                  ) for c in cat_cols_clas
]
cat_cols_clas_indexed = [c + '_indexed' for c in cat_cols_clas]   # Se crea una lista con los nombres de las columnas indexadas.
print(cat_cols_clas_indexed)

['color_indexed', 'clarity_indexed']


#### 6.2.2.-Imputers.

##### 6.2.2.1.-Imputers de variables numéricas.

In [26]:
imputer_numericas_clas = Imputer(
    inputCols=num_cols_clas,                                # Nombre de las columnas numéricas a imputar.
    outputCols=[c + '_imputed' for c in num_cols_clas],     # Nombre de salida de las columnas imputadas
strategy='median'                                           # Se imputa con la mediana.
)
num_cols_clas_imputed = [c + '_imputed' for c in num_cols_clas]   # Se crea una lista con los nombres de las columnas imputadas.
print(num_cols_clas_imputed)

['carat_imputed', 'depth_imputed', 'table_imputed', 'price_imputed', 'x_imputed', 'y_imputed', 'z_imputed']


##### 6.2.2.2.-Imputers de variables categóricas.

In [27]:
imputer_categoricas_clas = Imputer(
    inputCols=cat_cols_clas_indexed,                             # Nombre de las columnas categóricas a imputar.
    outputCols=[c + '_imputed' for c in cat_cols_clas_indexed],  # Nombre de salida de las columnas imputadas.
strategy='mode'                                                  # Se imputa con la moda.
)
cat_cols_clas_indexed_imputed = [c + '_imputed' for c in cat_cols_clas_indexed]   # Se crea una lista con los nombres de las columnas imputadas.
print(cat_cols_clas_indexed_imputed)

['color_indexed_imputed', 'clarity_indexed_imputed']


##### 6.2.2.3.-Codificación de variables categóricas.

In [28]:
encoder_onehot_clas = [
    OneHotEncoder(inputCol=c, outputCol=c + '_onehot') 
    for c in cat_cols_clas_indexed_imputed
]
cat_cols_clas_onehot = [c + '_onehot' for c in cat_cols_clas_indexed_imputed]
print(cat_cols_clas_onehot)

['color_indexed_imputed_onehot', 'clarity_indexed_imputed_onehot']


#### 6.2.3.-Escalados.

<p>En este caso, para las clasificación, se va a utilizar el algoritmo MultiLayerPerceptronClassifier para generar el modelo. Este algoritmo, sí está basado en distancias. Por tanto, es más sensible a las diferentes escalas de los valores de las variables.</p>
<p>El método del escalado que se va a emplear es MinMaxScaler, a pesar de la presencia de outliers en el DataFrame.</p>

##### 6.2.3.1-Ensamblado de variables.

<p>Antes de aplicar el escalado, se aplica el ensamblado de las variables para unificar ambos tipos de variables, numéricas y categóricas, en un único vector.</p>

In [29]:
# Se crea una lista con el nombre de las columnas a ensamblar, que seran las columnas 'features'.
col_ensambladas_clas = [col for col in num_cols_clas_imputed] + \
                         [col for col in cat_cols_clas_onehot]
print(col_ensambladas_clas)

['carat_imputed', 'depth_imputed', 'table_imputed', 'price_imputed', 'x_imputed', 'y_imputed', 'z_imputed', 'color_indexed_imputed_onehot', 'clarity_indexed_imputed_onehot']


In [30]:
# Se ensamblan los nombres de las columnas numéricas y categóricas en una única columna 'features'.
assembler_clas = VectorAssembler(
    inputCols=col_ensambladas_clas,      # Nombre de las columnas a ensamblar.
    outputCol='features'            # Nombre de salida de la columna ensamblada.
)

##### 6.2.3.2-Escalado.

In [31]:
# Escalado de todas las 'features' utilizando MinMaxScaler.
scaler_clas = MinMaxScaler(inputCol="features", outputCol="scaled_features")

### 6.3.-Modelado, Pipelines y Evaluación.

#### 6.3.1.-Particionamiento de datos.

In [32]:
df_clas_train, df_clas_test = df_clasificacion.randomSplit([0.8, 0.2], seed=42)  # Se divide el DataFrame en Train y Test.

#### 6.3.2.-Configuración del modelo MultiLayerPerceptronClassifier.

El parámetro 'layers' define la arquitectura de la red:
- La capa de entrada debe tener el tamaño igual al número de features.
- La capa de salida debe tener el número de clases (para "cut", generalmente 5 categorías).
- Se definen capas ocultas (por ejemplo, una capa oculta con 10 neuronas).
<p>Para determinar el tamaño de la capa de entrada, se entrena un pipeline temporal hasta el ensamblador.</p>

In [33]:
temp_pipeline = Pipeline(stages = [
    *indexers_features_clas,
    imputer_categoricas_clas,
    *encoder_onehot_clas, 
    imputer_numericas_clas,
    assembler_clas
    ])
temp_model = temp_pipeline.fit(df_reg_train)  # Se puede usar el mismo train_reg
temp_df = temp_model.transform(df_reg_train)
first_row = temp_df.select("features").first()
input_size = len(first_row['features'])
print("Tamaño de la capa de entrada (número de features):", input_size)

# Suponiendo que "cut" tiene 5 clases (por ejemplo: Fair, Good, Very Good, Premium, Ideal)
layers = [input_size, 10, 5]  # [capa de entrada, capa oculta, capa de salida]

Tamaño de la capa de entrada (número de features): 20


#### 6.3.3.-Generación del modelo.

In [41]:
mlp = MultilayerPerceptronClassifier(featuresCol="scaled_features", labelCol='label',
                                     layers=layers, maxIter=100)

#### 6.3.4.-Generación del Pipeline.

In [42]:
pipeline_clasificacion = Pipeline(stages = [
    indexer_label_clas,          # Indexador de la columna objetivo.
    *indexers_features_clas,     # Indexadores de las columnas categóricas. (se coloca '*' para desempaquetar la lista de objetos).
    imputer_numericas_clas,      # Imputador de las columnas numéricas.
    imputer_categoricas_clas,    # Imputador de las columnas categóricas.
    *encoder_onehot_clas,        # OneHotEncoder de las columnas categóricas. (se coloca '*' para desempaquetar la lista de objetos).
    assembler_clas,              # Ensamblador de las columnas numéricas y categóricas.
    scaler_clas,                 # Escalador de las 'features'.
    mlp
])

#### 6.3.5.-Entrenamiento del modelo.

In [43]:
modelo_clasificacion = pipeline_clasificacion.fit(df_clas_train)

#### 6.3.6.-Predicción del modelo.

In [44]:
df_clasificacion_transformado = modelo_clasificacion.transform(df_clas_test)

<p>Como se ha mencionado anteriormente, para la regresión se utiliza el algoritmo RandomForestRegressor.</p>

#### 6.3.7.-Evaluación del modelo.

In [ ]:



evaluador_r2 = RegressionEvaluator(labelCol=label_col, predictionCol='prediction', metricName='r2')
r2 = evaluador_r2.evaluate(prediccion_reg)
evaluador_mae = RegressionEvaluator(metricName='mae')
mae = evaluador_mae.evaluate(prediccion_reg) 
evaluador_mse = RegressionEvaluator(metricName='mse')
mse = evaluador_mse.evaluate(prediccion_reg)
evaluador_rmse = RegressionEvaluator(labelCol=label_col, predictionCol='prediction', metricName='rmse')
rmse = evaluador_rmse.evaluate(prediccion_reg)

print("=== Evaluación del Modelo de Regresión (RandomForestRegressor con RobustScaler) ===")
print(f"R2: {r2}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

=== Evaluación del Modelo de Regresión (RandomForestRegressor con RobustScaler) ===
R2: 0.9773520052305722
MAE: 1981.4725284922758
MSE: 7849110.502337356
RMSE: 607.0337566183553
